# Применим к свежим данным

### Некоторое количество копипаста:

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns

%matplotlib inline

In [8]:
raw_data = pd.read_csv("data/Moscow_weather_preparsed.csv").drop(columns=["Unnamed: 0"])
raw_data.head()

,DATE,TEMP,TEMP_Count,DEWP,DEWP_Count,SLP,SLP_Count,STP,STP_Count,VISIB,...,MAX_is_missing,MIN_is_missing,PRCP_is_missing,SNDP_is_missing,Is_Fog,Is_Rain or Drizzle,Is_Snow or Ice Pellets,Is_Hail,Is_Thunder,Is_Tornado or Funnel Cloud
0,1936-12-31,16.0,4.0,9999.9,0.0,1022.9,4.0,9999.9,0.0,3.0,...,0,0,0,1,0,0,0,0,0,0
1,1937-01-01,33.5,4.0,9999.9,0.0,1020.9,4.0,9999.9,0.0,2.5,...,0,0,0,1,0,1,0,0,0,0
2,1937-01-02,28.8,4.0,9999.9,0.0,1020.4,4.0,9999.9,0.0,4.0,...,0,0,0,1,0,0,0,0,0,0
3,1937-01-03,29.0,4.0,9999.9,0.0,1013.2,4.0,9999.9,0.0,2.5,...,0,0,0,1,0,1,1,0,0,0
4,1937-01-04,30.0,4.0,9999.9,0.0,1013.2,4.0,9999.9,0.0,2.2,...,0,0,0,1,0,0,1,0,0,0


In [30]:
data = raw_data.drop(columns=['STP', 'STP_Count', 'MAX_Flag', 'MIN_Flag', 'PRCP_Flag'])
data.head()

,DATE,TEMP,TEMP_Count,DEWP,DEWP_Count,SLP,SLP_Count,VISIB,VISIB_Count,WDSP,...,MAX_is_missing,MIN_is_missing,PRCP_is_missing,SNDP_is_missing,Is_Fog,Is_Rain or Drizzle,Is_Snow or Ice Pellets,Is_Hail,Is_Thunder,Is_Tornado or Funnel Cloud
0,1936-12-31,16.0,4.0,9999.9,0.0,1022.9,4.0,3.0,4.0,9.0,...,0,0,0,1,0,0,0,0,0,0
1,1937-01-01,33.5,4.0,9999.9,0.0,1020.9,4.0,2.5,4.0,13.0,...,0,0,0,1,0,1,0,0,0,0
2,1937-01-02,28.8,4.0,9999.9,0.0,1020.4,4.0,4.0,4.0,999.9,...,0,0,0,1,0,0,0,0,0,0
3,1937-01-03,29.0,4.0,9999.9,0.0,1013.2,4.0,2.5,4.0,17.0,...,0,0,0,1,0,1,1,0,0,0
4,1937-01-04,30.0,4.0,9999.9,0.0,1013.2,4.0,2.2,4.0,13.3,...,0,0,0,1,0,0,1,0,0,0


In [32]:
goal_features = list(data.columns[-6:])
# попробуем оставить только адекватные фичи
# все Count скорее всего можно игнорировать
# пропущенные значения и так достаточно отделены от возможных данных
# work_features = ["TEMP", "DEWP", "SLP", "VISIB", "WDSP", "MXSPD", "GUST", "MAX", "MIN", "PRCP", "SNDP"]
work_features = ["TEMP", "DEWP", "SLP", "VISIB", "WDSP", "MXSPD", "GUST", "MAX", "MIN", "PRCP", "SNDP"]
# work_features += [s + "_is_missing" for s in work_features]

In [33]:
# давайте учитывать данные до 3 дней в прошлое
def make_data_prev(data, work_features, goal_features):
    max_days_back = 3
    data_prev = {}
    goal = data[goal_features]
    cur_data = data[work_features]
    cur_res = pd.DataFrame(index=data.index)
    data_size = data.shape[0]
    for i in range(1, max_days_back + 1):
        # индексы - боль
        cur_data = cur_data.drop(index=cur_data.index[-1]).set_index(np.arange(data_size - i))
        goal = goal.drop(index=goal.index[0]).set_index(np.arange(data_size - i))
        cur_res = cur_res.drop(index=cur_res.index[0]).set_index(np.arange(data_size - i))
        for feature in work_features:
            cur_res["{} {} day(s) ago".format(feature, i)] = cur_data[feature]
        data_prev[i] = (cur_res, goal)
    return data_prev

data_prev = make_data_prev(data, work_features, goal_features)

### Модели с оптимальными параметрами:

In [34]:
best_params = [{'max_depth': 2, 'n_estimators': 90},
               {'max_depth': 8, 'n_estimators': 50},
               {'max_depth': 8, 'n_estimators': 70},
               {'max_depth': 3, 'n_estimators': 5},
               {'max_depth': 8, 'n_estimators': 20},]

In [35]:
X, y = data_prev[3]

In [36]:
from sklearn.ensemble import RandomForestClassifier

models = [RandomForestClassifier(max_depth=best_params[i]['max_depth'],
                                 n_estimators=best_params[i]['n_estimators'],
                                 random_state=123) for i in range(len(best_params))]
for model, goal in zip(models, goal_features):
    model.fit(X, y[goal])

## Загрузим свежие данные

In [37]:
fresh_data_raw = pd.read_csv("data/Fresh_data.csv")
fresh_data_raw.head()

,STN---,WBAN,YEARMODA,TEMP,,DEWP,.1,SLP,.2,STP,...,WDSP,.5,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT,Unnamed: 22
0,276120,99999,20181201,12.4,8,4.3,8,1037.9,8,1016.8,...,2.6,6,3.9,999.9,18.0,4.8,0.00F,1.6,0,NaN
1,276120,99999,20181202,22.8,8,18.0,8,1023.0,8,1002.6,...,3.6,8,3.9,999.9,24.8*,15.8,0.12F,2.4,1000,NaN
2,276120,99999,20181203,23.7,8,17.3,8,1020.3,8,1000.1,...,2.2,7,3.9,999.9,25.3,20.8*,0.01F,2.4,1000,NaN
3,276120,99999,20181204,25.5,8,19.7,8,1020.2,8,1000.0,...,4.4,8,5.8,999.9,30.7*,20.3,0.01F,2.4,1000,NaN
4,276120,99999,20181205,31.2,8,28.4,8,1020.2,8,1000.3,...,3.2,8,3.9,999.9,33.6,27.9*,0.02F,2.0,1000,NaN


In [40]:
# ещё немного копипаста
adeq_cols = ['STN---', 'WBAN', 'DATE', 'TEMP', 'TEMP_Count', 'DEWP', 'DEWP_Count', 'SLP', 'SLP_Count',
             'STP', 'STP_Count', 'VISIB', 'VISIB_Count', 'WDSP', 'WDSP_Count', 'MXSPD', 'GUST', 'MAX',
             'MIN', 'PRCP', 'SNDP', 'FRSHTT']
fresh_data = pd.DataFrame(fresh_data_raw.drop(columns=['Unnamed: 22']).values, columns=adeq_cols)

flagged = ['MAX', 'MIN', 'PRCP']
flags = {chr(ord('A') + i) for i in range(9)} | {'*'}

for name in flagged:
    fresh_data[name + '_Flag'] = fresh_data[name].map(lambda s: s[-1] if s[-1] in flags else '_')
    fresh_data[name] = fresh_data[name].map(lambda s: s[:-1] if s[-1] in flags else s)

# есть ещё кучи девяток вместо nan-ов
missing = [9999.9, np.nan, 9999.9, np.nan, 9999.9, np.nan, 9999.9,
           np.nan, 999.9, np.nan, 999.9, np.nan, 999.9, 999.9, 9999.9, 9999.9, 99.9, 999.9]
for miss, name in zip(missing, fresh_data.columns[3:]):
    fresh_data[name] = fresh_data[name].astype(float)
    fresh_data[name + '_is_missing'] = fresh_data[name].map(lambda s: int(s == miss))
    
# последняя волшебная колонка
FRSHTT = ['Fog', 'Rain or Drizzle', 'Snow or Ice Pellets', 'Hail', 'Thunder', 'Tornado or Funnel Cloud']
for i in range(len(FRSHTT)):
    fresh_data['Is_' + FRSHTT[i]] = fresh_data['FRSHTT'].map(lambda s: ((int(str(s), 2) >> (len(FRSHTT) - i - 1)) & 1))
fresh_data = fresh_data.drop(columns=['FRSHTT'])

# первые колонки нафиг не нужны, они полностью одинаковые
data = data.drop(columns=data.columns[:2])

def f(v):
    v = str(v)
    y, m, d = int(v[:4]), int(v[4:6]), int(v[6:])
    return pd.datetime(y, m, d)

fresh_data['DATE'] = fresh_data['DATE'].map(f)

fresh_data

,STN---,WBAN,DATE,TEMP,TEMP_Count,DEWP,DEWP_Count,SLP,SLP_Count,STP,...,MAX_is_missing,MIN_is_missing,PRCP_is_missing,SNDP_is_missing,Is_Fog,Is_Rain or Drizzle,Is_Snow or Ice Pellets,Is_Hail,Is_Thunder,Is_Tornado or Funnel Cloud
0,276120,99999,2018-12-01,12.4,8.0,4.3,8.0,1037.9,8.0,1016.8,...,0,0,0,0,0,0,0,0,0,0
1,276120,99999,2018-12-02,22.8,8.0,18.0,8.0,1023.0,8.0,1002.6,...,0,0,0,0,0,0,1,0,0,0
2,276120,99999,2018-12-03,23.7,8.0,17.3,8.0,1020.3,8.0,1000.1,...,0,0,0,0,0,0,1,0,0,0
3,276120,99999,2018-12-04,25.5,8.0,19.7,8.0,1020.2,8.0,1000.0,...,0,0,0,0,0,0,1,0,0,0
4,276120,99999,2018-12-05,31.2,8.0,28.4,8.0,1020.2,8.0,1000.3,...,0,0,0,0,0,0,1,0,0,0
5,276120,99999,2018-12-06,29.9,8.0,28.0,8.0,1019.6,8.0,999.6,...,0,0,0,0,0,0,1,0,0,0
6,276120,99999,2018-12-07,26.5,8.0,21.9,8.0,1018.8,8.0,998.7,...,0,0,0,0,0,0,1,0,0,0
7,276120,99999,2018-12-08,24.7,8.0,20.1,8.0,1015.6,8.0,995.4,...,0,0,0,0,0,0,0,0,0,0
8,276120,99999,2018-12-09,25.4,8.0,23.0,8.0,1009.5,8.0,989.5,...,0,0,0,0,0,0,1,0,0,0
9,276120,99999,2018-12-10,33.2,8.0,31.3,8.0,1007.4,8.0,987.8,...,0,0,0,0,0,0,1,0,0,0


In [39]:
fresh_data_prev = make_data_prev(fresh_data, work_features, goal_features)
fresh_data_prev[3][0]

,TEMP 1 day(s) ago,DEWP 1 day(s) ago,SLP 1 day(s) ago,VISIB 1 day(s) ago,WDSP 1 day(s) ago,MXSPD 1 day(s) ago,GUST 1 day(s) ago,MAX 1 day(s) ago,MIN 1 day(s) ago,PRCP 1 day(s) ago,...,DEWP 3 day(s) ago,SLP 3 day(s) ago,VISIB 3 day(s) ago,WDSP 3 day(s) ago,MXSPD 3 day(s) ago,GUST 3 day(s) ago,MAX 3 day(s) ago,MIN 3 day(s) ago,PRCP 3 day(s) ago,SNDP 3 day(s) ago
0,23.7,17.3,1020.3,999.9,2.2,3.9,999.9,25.3,20.8,0.01,...,4.3,1037.9,999.9,2.6,3.9,999.9,18.0,4.8,0.00,1.6
1,25.5,19.7,1020.2,999.9,4.4,5.8,999.9,30.7,20.3,0.01,...,18.0,1023.0,999.9,3.6,3.9,999.9,24.8,15.8,0.12,2.4
2,31.2,28.4,1020.2,999.9,3.2,3.9,999.9,33.6,27.9,0.02,...,17.3,1020.3,999.9,2.2,3.9,999.9,25.3,20.8,0.01,2.4
3,29.9,28.0,1019.6,999.9,2.3,3.9,999.9,31.8,27.5,0.35,...,19.7,1020.2,999.9,4.4,5.8,999.9,30.7,20.3,0.01,2.4
4,26.5,21.9,1018.8,999.9,2.5,3.9,999.9,29.7,24.1,0.01,...,28.4,1020.2,999.9,3.2,3.9,999.9,33.6,27.9,0.02,2.0
5,24.7,20.1,1015.6,999.9,2.4,3.9,999.9,25.9,22.5,0.00,...,28.0,1019.6,999.9,2.3,3.9,999.9,31.8,27.5,0.35,5.1
6,25.4,23.0,1009.5,999.9,3.2,3.9,999.9,28.0,23.4,0.13,...,21.9,1018.8,999.9,2.5,3.9,999.9,29.7,24.1,0.01,5.5
7,33.2,31.3,1007.4,999.9,2.5,3.9,999.9,34.9,26.4,0.09,...,20.1,1015.6,999.9,2.4,3.9,999.9,25.9,22.5,0.00,5.5
8,33.2,32.1,1012.3,999.9,2.2,3.9,999.9,34.0,32.4,0.22,...,23.0,1009.5,999.9,3.2,3.9,999.9,28.0,23.4,0.13,5.5
9,32.9,30.6,1019.1,999.9,3.2,5.8,999.9,34.2,32.0,0.00,...,31.3,1007.4,999.9,2.5,3.9,999.9,34.9,26.4,0.09,6.7


### Попробуем предсказать погоду

In [44]:
for model, goal in zip(models, goal_features):
    print("Predicting {}:".format(goal))
    res = model.predict(fresh_data_prev[3][0])
    corr = fresh_data_prev[3][1][goal]
    print(*res)
    print(*corr)
    print("Score is:", model.score(fresh_data_prev[3][0], corr))
    print()

Predicting Is_Fog:
0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0
Score is: 1.0

Predicting Is_Rain or Drizzle:
0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0
Score is: 1.0

Predicting Is_Snow or Ice Pellets:
1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1
Score is: 0.909090909091

Predicting Is_Hail:
0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0
Score is: 1.0

Predicting Is_Thunder:
0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0
Score is: 1.0



Ну вот, пропустили снег в один день... Как бы декабрь предсказывать довольно безысходно.